In [1]:
# Importing packages
import pandas as pd
import sqlite3
import os

In [2]:
# Loading in the cleaned data
df = pd.read_csv("data/cwur_2025_cleaned.csv", encoding="utf-8")

In [3]:
# I would like to keep all saved data in the same folder
os.makedirs("data", exist_ok=True)

In [4]:
# Defining database filename and table name
db_filename = "data/rankings.db"
table_name = "university_rankings"

In [5]:
# Connecting to the SQLite database
conn = sqlite3.connect(db_filename)

In [6]:
# Storing the DataFrame into the database
df.to_sql(
    name=table_name,
    con=conn,
    if_exists="replace",  
    index=False           # ensuring to not store the DataFrame index as a column
)

# Commiting and closing the connection
conn.commit()
conn.close()

print(f"Data successfully stored in '{db_filename}' as table '{table_name}'.")

Data successfully stored in 'data/rankings.db' as table 'university_rankings'.


In [ ]:
# # I keep getting errors later in the parquet, this is a recommended change I saw online
# df_adj = df.convert_dtypes()
# df_adj = df_adj.astype({col: "float64" for col in df_adj.select_dtypes(include="Int64").columns})
# commented out, didn't need to do this afterall

# Storing the clean DataFrame in Parquet format, in the same data file as before
parquet_filename = "data/rankings.parquet"

In [8]:
# importing required package
import fastparquet

# Saving to parquet
df.to_parquet(
    parquet_filename,
    index=False,              # Ensuring to not store the DataFrame index
    engine="fastparquet",         
    compression="snappy"      # Default compression, but efficient
)

print("DataFrame successfully stored as Parquet.")

DataFrame successfully stored as Parquet.


In [12]:
def validate_data_integrity(df_original, db_path, table_name, parquet_path):
    """
    Reads data back from SQLite and Parquet, and compares them to the cleaned DataFrame from before.
    """

    # Read from SQLite
    with sqlite3.connect(db_path) as conn:
        df_sqlite = pd.read_sql(f"SELECT * FROM {table_name}", conn)

    # Read from Parquet 
    df_parquet = pd.read_parquet(parquet_path)

    # Compare dataframes
    same_sqlite = df_original.equals(df_sqlite)
    same_parquet = df_original.equals(df_parquet)

    print("Data validation results:")
    print(f"SQLite  match: {same_sqlite}")
    print(f"Parquet match: {same_parquet}")

    # Optional: return the loaded DataFrames for further inspection
    return df_sqlite, df_parquet


df_sqlite, df_parquet = validate_data_integrity(df, db_filename, table_name, parquet_filename)

Data validation results:
SQLite  match: True
Parquet match: True


In [10]:
# previewingp arquet data
df_parquet.head()

,Rank Number,Institution,Country,National Rank,Education Rank,Employability Rank,Faculty Rank,Research Rank,Score,Global_Region,Overall_Score_Normalized
0,1,Harvard University,USA,1,1.0,1.0,1.0,1.0,100.0,America,1.000000
1,2,Massachusetts Institute of Technology,USA,2,4.0,12.0,2.0,11.0,96.8,America,0.905325
2,3,Stanford University,USA,3,10.0,4.0,3.0,4.0,95.2,America,0.857988
3,4,University of Cambridge,United Kingdom,1,2.0,26.0,4.0,14.0,94.1,Europe,0.825444
4,5,University of Oxford,United Kingdom,2,7.0,28.0,9.0,6.0,93.3,Europe,0.801775


In [11]:
# previewingp sqlite data
df_sqlite.head()

,Rank Number,Institution,Country,National Rank,Education Rank,Employability Rank,Faculty Rank,Research Rank,Score,Global_Region,Overall_Score_Normalized
0,1,Harvard University,USA,1,1.0,1.0,1.0,1.0,100.0,America,1.000000
1,2,Massachusetts Institute of Technology,USA,2,4.0,12.0,2.0,11.0,96.8,America,0.905325
2,3,Stanford University,USA,3,10.0,4.0,3.0,4.0,95.2,America,0.857988
3,4,University of Cambridge,United Kingdom,1,2.0,26.0,4.0,14.0,94.1,Europe,0.825444
4,5,University of Oxford,United Kingdom,2,7.0,28.0,9.0,6.0,93.3,Europe,0.801775
